<a href="https://colab.research.google.com/github/VipanchiKatthula/PySpark_OperationsOnAmazonTweets/blob/master/PySpark_Operations_on_Amazon_Tweets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null   #Install Spark

In [0]:
!wget -q http://apache.mirrors.pair.com/spark/spark-2.4.5/spark-2.4.5-bin-hadoop2.7.tgz  #Get Spark Insatller

In [0]:
!tar -xvf spark-2.4.5-bin-hadoop2.7.tgz  #Untar Spark installer

***Install findspark - a python library to find Spark***

In [0]:
!pip install -q findspark   

***Setting environmental Variables***

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.5-bin-hadoop2.7"

***Create a local Spark session***

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

***Importing data and Selecting required columns***

In [0]:
data = spark.read.format("csv").option("header","true").load("drive/My\ Drive/Colab\ Notebooks/Data/Amazon_Responded_Oct05.csv")
data1 = data.select("id_str",'tweet_created_at','user_verified','favorite_count','retweet_count','text')

In [18]:
data1.count() #Number of rows in data

413247

In [189]:
data1.columns

['id_str',
 'tweet_created_at',
 'user_verified',
 'favorite_count',
 'retweet_count',
 'text']

In [190]:
types = [f.dataType for f in data1.schema.fields]
types

[StringType, StringType, StringType, StringType, StringType, StringType]

In [191]:
dat_filtered = data1.filter(data1.user_verified == 'True')
dat_filtered.count() #Selecting only those tweets with vaild/verified users

171797

In [192]:
dat_filtered.select("user_verified").distinct().show()

+-------------+
|user_verified|
+-------------+
|         True|
+-------------+



In [193]:
dat_filtered.select("tweet_created_at").show(5,False)

+------------------------------+
|tweet_created_at              |
+------------------------------+
|Tue Nov 01 02:39:55 +0000 2016|
|Tue Nov 01 17:19:57 +0000 2016|
|Tue Nov 01 17:25:26 +0000 2016|
|Tue Nov 01 18:02:03 +0000 2016|
|Tue Nov 01 03:59:02 +0000 2016|
+------------------------------+
only showing top 5 rows



In [194]:
from pyspark.sql.functions import split
a = split(dat_filtered["tweet_created_at"], ' ')
dat_filtered1 = dat_filtered.withColumn('Month', a.getItem(1))
dat_filtered1 = dat_filtered1.withColumn('Date', a.getItem(2))
dat_filtered1 = dat_filtered1.withColumn('Year', a.getItem(5))
dat_filtered1.show(5)


+--------------------+--------------------+-------------+--------------+-------------+--------------------+-----+----+----+
|              id_str|    tweet_created_at|user_verified|favorite_count|retweet_count|                text|Month|Date|Year|
+--------------------+--------------------+-------------+--------------+-------------+--------------------+-----+----+----+
|'793281386912354304'|Tue Nov 01 02:39:...|         True|             0|            0|                null|  Nov|  01|2016|
|'793502854459879424'|Tue Nov 01 17:19:...|         True|             0|            0|@SeanEPanjab Plea...|  Nov|  01|2016|
|'793504235400884224'|Tue Nov 01 17:25:...|         True|             0|            0|                null|  Nov|  01|2016|
|'793513446633533440'|Tue Nov 01 18:02:...|         True|             0|            0|@SeanEPanjab I'm ...|  Nov|  01|2016|
|'793301295255945216'|Tue Nov 01 03:59:...|         True|             0|            0|                null|  Nov|  01|2016|
+-------

In [195]:
a = dat_filtered1.select(concat(col("Month"), lit(" "), col("Date"),lit(" "), col("Year")).alias("Date"))
a.show(5)

+-----------+
|       Date|
+-----------+
|Nov 01 2016|
|Nov 01 2016|
|Nov 01 2016|
|Nov 01 2016|
|Nov 01 2016|
+-----------+
only showing top 5 rows



In [208]:
import pyspark.sql.functions as sq 
#from pyspark.sql import functions as sf

df = dat_filtered1.withColumn("tweet_created_at",sq.concat(col("Month"), sq.lit(" "), sq.col("Date"),sq.lit(" "), sq.col("Year")))
df = df.select('id_str','tweet_created_at','user_verified',df.favorite_count.cast('int'), df.retweet_count.cast('int'),'text')
df.show(5)

+--------------------+----------------+-------------+--------------+-------------+--------------------+
|              id_str|tweet_created_at|user_verified|favorite_count|retweet_count|                text|
+--------------------+----------------+-------------+--------------+-------------+--------------------+
|'793281386912354304'|     Nov 01 2016|         True|             0|            0|                null|
|'793502854459879424'|     Nov 01 2016|         True|             0|            0|@SeanEPanjab Plea...|
|'793504235400884224'|     Nov 01 2016|         True|             0|            0|                null|
|'793513446633533440'|     Nov 01 2016|         True|             0|            0|@SeanEPanjab I'm ...|
|'793301295255945216'|     Nov 01 2016|         True|             0|            0|                null|
+--------------------+----------------+-------------+--------------+-------------+--------------------+
only showing top 5 rows



In [209]:
types = [f.dataType for f in df.schema.fields]
types

[StringType, StringType, StringType, IntegerType, IntegerType, StringType]

In [206]:
counts = df.groupby(df.tweet_created_at).agg(sq.count('id_str').alias("count_of_tweets"))
counts.show(5)

+----------------+---------------+
|tweet_created_at|count_of_tweets|
+----------------+---------------+
|     Dec 01 2016|            875|
|     Dec 25 2016|            433|
|     Feb 08 2017|            926|
|     Feb 19 2017|            725|
|     Mar 30 2017|           1109|
+----------------+---------------+
only showing top 5 rows



In [210]:
conf = pyspark.SparkConf()
sc = pyspark.SparkContext.getOrCreate(conf=conf)
from pyspark.sql import SQLContext
sqlcontext = SQLContext(sc)

counts.registerTempTable("tmpcounts")
counts_ordered = sqlcontext.sql("SELECT * FROM tmpcounts order by count_of_tweets desc limit 5")
counts_ordered.show()

+----------------+---------------+
|tweet_created_at|count_of_tweets|
+----------------+---------------+
|     Jan 03 2017|           1536|
|     Jan 10 2017|           1508|
|     Jan 11 2017|           1496|
|     Jan 12 2017|           1410|
|     Jan 06 2017|           1364|
+----------------+---------------+



In [227]:
df.registerTempTable("tmpdf")
#df2 = df.withColumn("favourite_count", "favourite_count" cast "Int")
sum_retweets = sqlcontext.sql("SELECT text,favorite_count+ retweet_count as total from tmpdf where tweet_created_at = 'Jan 03 2017' order by total desc limit 100")
sum_retweets.show(10,False)

+--------------------------------------------------------------------------------------------------------------------------------------------+-----+
|text                                                                                                                                        |total|
+--------------------------------------------------------------------------------------------------------------------------------------------+-----+
|null                                                                                                                                        |5    |
|null                                                                                                                                        |3    |
|@ItsJosshA Oh no! I'm sorry! Please reach out to us so that we can look into options: https://t.co/hApLpMlfHN. ^JO                          |2    |
|null                                                                                                     

***Task -2***

In [230]:
finddata = spark.read.format("csv").option("header","true").load("drive/My\ Drive/Colab\ Notebooks/Data/find_text.csv")
finddata.show(5,False)

+--------------------+----+
|id_str              |text|
+--------------------+----+
|'793270689780203520'|null|
|'793281386912354304'|null|
|'793299404975247360'|null|
|'793301295255945216'|null|
|'793315815411978240'|null|
+--------------------+----+
only showing top 5 rows



In [237]:
finddata.registerTempTable("ids")
data.registerTempTable("tweets")

out = sqlcontext.sql("SELECT DISTINCT I.id_str,T.text from ids I JOIN tweets T on I.id_str = T.id_str")
out.show(5,False)

+--------------------+-----------------------------------------------------------------------------------------------------------------------------------------+
|id_str              |text                                                                                                                                     |
+--------------------+-----------------------------------------------------------------------------------------------------------------------------------------+
|'793382930085253121'|@mybharatraj Hi! Sorry about that. I'd like our team to look into this, please reach out to us via: https://t.co/vlvfJr4nN9 ^SG          |
|'793533066157387776'|@flamablebrownie When checking your account, are you signed into the email that was used to place the order: https://t.co/aaDyEz1VgE? ^HB|
|'793552243689676800'|null                                                                                                                                     |
|'793595479116439552'|null        

In [238]:
print(finddata.count(), data.count(),out.count())

53928 413247 53927
